<a href="https://colab.research.google.com/github/santosh50/Text_to_Emoji/blob/main/emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
mapping = pd.read_csv("/content/drive/My Drive/NLP/Mapping.csv")
train = pd.read_csv("/content/drive/My Drive/NLP/Train_balanced.csv")
test = pd.read_csv("/content/drive/My Drive/NLP/Test.csv")
out = pd.read_csv("/content/drive/My Drive/NLP/OutputFormat.csv")

In [27]:
mapping = mapping.drop(['Unnamed: 0'], axis = 1)

In [28]:
from sklearn.utils import resample
train=resample(train,replace=False,n_samples=1000,random_state=123)

In [29]:
emoticons = mapping['emoticons'].tolist()

In [30]:
mapp = {}
for emo in range(0,len(emoticons)):
    mapp[emo]= emoticons[emo]

In [31]:
mapp

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [10]:
train.head()


,Unnamed: 0,text,label
43073,67567,Ladies night.... #funtimes @ MKT BAR\n,8
34567,49424,Happiness depends upon ourselves #california #...,11
25967,31239,#SWEETCHICKBLOCKPARTY #WILLY.B #BLEUMARLI #PAP...,1
8850,60845,Date night at Mo's w my baby. @ Moe's Place\n,2
26616,32623,Thanks for the best day fam besides the part w...,14


In [32]:

train = train.drop(['Unnamed: 0'], axis = 1)
test = test.drop(['Unnamed: 0'], axis = 1)


In [ ]:
train

,text,label
43073,Ladies night.... #funtimes @ MKT BAR\n,8
34567,Happiness depends upon ourselves #california #...,11
25967,#SWEETCHICKBLOCKPARTY #WILLY.B #BLEUMARLI #PAP...,1
8850,Date night at Mo's w my baby. @ Moe's Place\n,2
26616,Thanks for the best day fam besides the part w...,14
...,...,...
22469,W KEND: Count your blessings. You are one of a...,11
35625,First workout of the year is done! (@ Train Fo...,10
44182,Sending sweet dreams from one of the dreamiest...,6
17388,It's always a good day when you get to selfie ...,16


In [33]:
x_train = train['text'].values
y_train = train['label'].values
x_text = train['text'].values
x_test = test['TEXT'].values

In [34]:
x_train.shape, x_test.shape

((1000,), (25958,))

## embed

In [35]:
f = open("/content/drive/My Drive/NLP/glove.6B.50d.txt", encoding='utf8')

In [36]:
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    emb = np.array(values[1:], dtype ='float')
    embedding_index[word] = emb

In [37]:
embedding_index['america'].shape

(50,)

In [38]:
def get_embedding_output(X):
    maxLen = 20
    embedding_output = np.zeros((len(X), maxLen, 50))
    
    for ix in range(X.shape[0]):
        my_example = X[ix].split()
        
#         print(my_example)       
        for ij in range(len(my_example)): 
            if (embedding_index.get(my_example[ij].lower()) is not None) and (ij<maxLen):
                embedding_output[ix][ij] = embedding_index[my_example[ij].lower()]
            
    return embedding_output

In [39]:
x_train_embed = get_embedding_output(x_train)

In [40]:
x_train_embed.shape

(1000, 20, 50)

In [41]:
from keras.utils import to_categorical

In [42]:
y_train = to_categorical(y_train)

In [29]:
y_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

## SMOTE

In [43]:
from imblearn.over_sampling import SMOTE

In [45]:
x_train_embed.shape

(1000, 1000)

In [44]:
x_train_embed = x_train_embed.reshape(-1, 1000)

In [46]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [47]:
oversample = SMOTE()
X, y = oversample.fit_resample(x_train_embed, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [48]:
X = X.reshape(-1, 20, 50)
X.shape, y.shape

((1440, 20, 50), (1440, 20))

## Split

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
x1, x2, y1, y2 = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# for i in range(0, 5):
#     print(x_train[i], mapp[y_train[i]])
x1.shape

(1152, 20, 50)

In [52]:
y1.shape

(1152, 20)

## *LSTM*

In [53]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [54]:
model = Sequential()
model.add(Bidirectional(LSTM(units = 512, return_sequences=True),input_shape = (20,50)))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(units=256)))
model.add(Dropout(0.4))
#model.add(Dense(units=128, activation='relu'))
#model.add(Dense(units=64, activation='relu'))
#model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=20, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 20, 1024)          2306048   
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 1024)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               2623488   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                10260     
Total params: 4,939,796
Trainable params: 4,939,796
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [57]:
hist = model.fit(x1, y1, validation_split=0.2, shuffle=True, batch_size=32, epochs=25)

Epoch 1/25
29/29 [==============================] - 32s 1s/step - loss: 2.9867 - accuracy: 0.0869 - val_loss: 2.9320 - val_accuracy: 0.1385
Epoch 2/25
29/29 [==============================] - 32s 1s/step - loss: 2.8051 - accuracy: 0.1759 - val_loss: 2.8360 - val_accuracy: 0.1299
Epoch 3/25
29/29 [==============================] - 32s 1s/step - loss: 2.6416 - accuracy: 0.2269 - val_loss: 2.8896 - val_accuracy: 0.1515
Epoch 4/25
29/29 [==============================] - 31s 1s/step - loss: 2.4675 - accuracy: 0.2606 - val_loss: 2.8562 - val_accuracy: 0.1688
Epoch 5/25
29/29 [==============================] - 32s 1s/step - loss: 2.2322 - accuracy: 0.3290 - val_loss: 2.7844 - val_accuracy: 0.1861
Epoch 6/25
29/29 [==============================] - 32s 1s/step - loss: 2.0541 - accuracy: 0.3844 - val_loss: 2.8195 - val_accuracy: 0.1991
Epoch 7/25
29/29 [==============================] - 32s 1s/step - loss: 1.7898 - accuracy: 0.4582 - val_loss: 2.6897 - val_accuracy: 0.2381
Epoch 8/25
29/29 [==

In [58]:
model.evaluate(x2, y2)

9/9 [==============================] - 3s 200ms/step - loss: 3.4073 - accuracy: 0.4236


[3.407332420349121, 0.4236111044883728]

In [ ]:
import random
pred=model.predict(x2)[:]
pred=np.argmax(pred, axis = 1)[:]
label=np.argmax(y2, axis = 1)[:]
l=random.sample(range(0,len(x2)),20)
for i in l:
  print(x2[i],mapp[pred[i]],mapp[label[i]])
  

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 5.29826056e-01  3.36298105e-01 -2.83702983e-01  2.75556772e-02
   4.46892964e-01  6.82925545e-02 -3.60524300e-01 -2.09038379e-01
  -1.58923424e-01  1.08923816e-01 -1.6027